##### 2021/07
###### author : 손해원

- 인스타그램에 키워드를 검색하여 게시글의 정보를 크롤링하는 코드입니다. 

In [1]:
from selenium import webdriver
import time
import re
import json
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
import os
import multiprocessing

### Load Chromedriver & Log In

In [2]:
my_id = 'yeonhhhhh5'   ### ID 
my_pw = '@aa1149118'   ### PASSWORD
driver_path = 'C:\dev_python\Webdriver\chromedriver'
instagram ='https://www.instagram.com'
# id/pw 입력 -> 로그인 정보 저장 여부 -> 피드

In [3]:
driver = webdriver.Chrome(driver_path)
driver.get(instagram)

<ipython-input-3-2328cdddebf4>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)


In [4]:
time.sleep(2)
id_ = driver.find_element_by_css_selector('#loginForm > div > div:nth-child(1) > div > label > input') # id
id_.send_keys(my_id)
pw_ = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input') # pw
pw_.send_keys(my_pw)
driver.find_element_by_css_selector('#loginForm > div > div:nth-child(3)').click() # 로그인


<ipython-input-4-26b57dbbb05f>:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  id_ = driver.find_element_by_css_selector('#loginForm > div > div:nth-child(1) > div > label > input') # id
<ipython-input-4-26b57dbbb05f>:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  pw_ = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input') # pw
<ipython-input-4-26b57dbbb05f>:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('#loginForm > div > div:nth-child(3)').click() # 로그인


In [6]:
time.sleep(3)
driver.find_element_by_class_name('sqdOP.yWX7d.y3zKF').click() # 정보 저장 안함
    
driver.wait = driver.implicitly_wait(10)

# 알림 설정 나중에 
driver.find_element_by_css_selector('body > div.RnEpo.Yx5HN > div > div > div > div.mt3GC > button.aOOlW.HoLwm').click()
    

<ipython-input-6-31d05c215dbb>:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_class_name('sqdOP.yWX7d.y3zKF').click() # 정보 저장 안함
<ipython-input-6-31d05c215dbb>:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('body > div.RnEpo.Yx5HN > div > div > div > div.mt3GC > button.aOOlW.HoLwm').click()


In [7]:
query = '김천숙박'   ######################## 검색할 내용 입력

try:
    url = 'https://www.instagram.com/explore/tags/{}/'.format(query)
    driver.get(url)
except :
    time.sleep(3)
    driver.get(url) 
    
driver.maximize_window()

In [8]:
# 첫번째 게시물 클릭
first_img = '_9AhH0'
driver.find_element_by_class_name(first_img).click()

<ipython-input-8-0c229527d784>:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_class_name(first_img).click()


### 게시물 스크랩

In [9]:
from urllib.request import urlopen
from urllib.parse import quote_plus
from bs4 import BeautifulSoup
import requests
import shutil

In [10]:
# data lists
cnt = []               #Content_Counter 
content_links = []     #Content_Link 
user_ids = []          
location_infos = []
location_hrefs = []
main_texts = []
tags= []
likes = []
who_likes = []
date_texts = []
date_times = []
date_titles = []

tag_lists = []
img_list = []


# path 설정
user_id_path = 'e1e1d'
location_class = 'O4GlU'
main_text_path = 'C4VMK'
tags_class_name = ' xil3i'
likes_path = 'zV_Nj'
date_path = '_1o9PC.Nzb55'
next_path = '_65Bje.coreSpriteRightPaginationArrow'


wish_num = 70# 스크랩할 게시물 개수 설정 
num = 0


for num in tqdm(range(wish_num)):

    # sequence
    cnt.append(num)

    # 글 링크
    try:
        time.sleep(1)
        content_link = driver.find_element_by_class_name('zV_Nj').get_attribute('href')
        content_links.append(content_link)
    except:
        content_links.append("NULL")

    # user ID
    try :
        user_id_obj = driver.find_element_by_class_name(user_id_path)
        user_id = user_id_obj.text

        user_ids.append(user_id)

    except :
        user_ids.append("NULL")


    # 위치정보
    try :
        location_obj = driver.find_element_by_class_name(location_class)
        location_text = location_obj.text
        location_href = location_obj.get_attribute("href")

        location_infos.append(location_text)
        location_hrefs.append(location_href)
    except :
        location_infos.append("NULL")
        location_hrefs.append("NULL")


    # 이미지 링크 
    try:
        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html)
        instagram = soup.select('.v1Nh3.kIKUG._bz0w')
        img_url = instagram[num].select_one('.KL4Bh').img['src']
        img_list.append(img_url)
        

    except:
        img_list.append("NULL")
        

    # 본문
    try :
        main_text_obj = driver.find_element_by_class_name(main_text_path)
        main_text = main_text_obj.text
        clean_text = main_text.replace("\n"," ")
        #comments = driver.find_element_by_class_name('Mr508 ').text.replace('\n',' ')

        main_texts.append(clean_text)

    except :
        main_texts.append("NULL")

    # 해쉬태그
    try:
        tag_list = []
        tags = driver.find_elements_by_class_name('xil3i')
        for i in range(len(tags)):
            tag = tags[i].text.replace('#', '')
            tag_list.append(tag)
        tag_lists.append(tag_list)
    except:
        tag_list.append("NULL")

    # 좋아요 개수
    try :
        time.sleep(2)
        like = driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/a/span')
    
        likee = like.text
        likes.append(likee)
    except :
        likes.append(None)


    # 날짜
    #try :
    date_obj = driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[2]/div/div/div[2]/div[2]/a/time')
    
    date_text = date_obj.get_attribute("title")
        
    date_texts.append(date_text)
        #date_time = date_obj.get_attribute("datetime")
        #date_title = date_obj.get_attribute("title") # 2021년 7월 14일
        
        #date_times.append(date_time)
        #date_titles.append(date_title)
   # except :
        #date_texts.append("NULL")
        #date_times.append("NULL")
        #date_titles.append("NULL")


    print(" ")
    print(f"-----{num}-----")
    print("ID: ", user_ids[num])
    print("위치정보: ", location_infos[num])
    print("내용: ", main_texts[num])
    print("날짜: ", date_texts[num])
    print(" 좋아요개수 :",likes[num])
    print("   ")


    # 다음 페이지 여부 확인
    try:
        next_level = driver.find_element_by_class_name('l8mY4 ')
        next_level.click()
        print('clicked')
        num += 1
    except:
        break
        


  0%|                                                                                           | 0/70 [00:00<?, ?it/s]<ipython-input-10-c2e964e87a65>:41: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  content_link = driver.find_element_by_class_name('zV_Nj').get_attribute('href')
<ipython-input-10-c2e964e87a65>:48: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  user_id_obj = driver.find_element_by_class_name(user_id_path)
<ipython-input-10-c2e964e87a65>:59: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  location_obj = driver.find_element_by_class_name(location_class)
<ipython-input-10-c2e964e87a65>:86: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  main_text_obj = driver.find_element_by_class_name(main_text_path)
<ipython-input-10-c2e964e87a65>:99: DeprecationWarning: find_element

 
-----0-----
ID:  solchan_i
위치정보:  Gimcheon
내용:  solchan_i 🏡 오늘의 손님 맞이, 편안한 쉼_이 되시길🎀 . . #에어비앤비 #솔과밤 #김천여행 #김천숙소 #김천숙박 #팜스테이 5일
날짜:  2021년 11월 5일
 좋아요개수 : 55
   


  1%|█▏                                                                                 | 1/70 [00:06<07:39,  6.66s/it]

clicked


  3%|██▎                                                                                | 2/70 [00:22<13:51, 12.22s/it]

...

 51%|██████████████████████████████████████████▏                                       | 36/70 [07:54<09:31, 16.80s/it]

clicked


 53%|███████████████████████████████████████████▎                                      | 37/70 [08:10<09:09, 16.65s/it]

 
-----36-----
ID:  coffeeblossom7
위치정보:  NULL
내용:  coffeeblossom7 접시에 쿠키 담다가 나온 게스트 간판  처음 열정을 잊을때마다 기분좋은 묘약이 되어줍니다^^ #김천게스트하우스 #김천숙박 #김천숙소#airbnb #blossomguesthouse#f4f #김천#교동#연화지#로스터리카페#커피블럭 #블러썸게스트하우스 237주
날짜:  2017년 4월 20일
 좋아요개수 : 147
   
clicked
 
-----37-----
ID:  seonjinkong
위치정보:  NULL
내용:  seonjinkong 즐거운명절! 행복하고 여유롭게 즐기세요! #추석 #잠자리 #김천숙박 #로제니아호텔 #김천호텔 #혁신도시호텔 #추석이벤트 271주
날짜:  2016년 8월 26일
 좋아요개수 : 6
   


 53%|███████████████████████████████████████████▎                                      | 37/70 [08:36<07:40, 13.96s/it]


In [11]:
len(date_titles)

0

In [12]:
instagram_df=pd.DataFrame(
    {
        "cnt": cnt,
        "link":content_links,
        "id": user_ids
        , "location_info": location_infos
        , "location_href": location_hrefs
        , "main_text": main_texts
        , "hastag" : tag_lists
        , "like": likes
        #, "date_time": date_times
        , "date_title": date_texts
    }
)

In [13]:
# dfdf = pd.concat([instagram_df_1, instagram_df_2] , axis=0)

# instagram_df_ok = pd.concat([dfdf, instagram_df_3], axis=0)

In [14]:
instagram_df.all()

cnt              False
link              True
id                True
location_info     True
location_href     True
main_text         True
hastag            True
like              True
date_title        True
dtype: bool

In [15]:
#instagram_df.to_csv('./instagram_seoul_1102.csv', index=False)

#### 좋아요 목록의 유저 링크 가져오기

In [16]:
###### 0831 완셩

def get_likers():
    real_list = []

    for num in tqdm(range(len(content_links))): # 컨텐츠 링크를 읽어들여 그 수만큼 반복

        url = content_links[num]
        driver.get(url)

        like_obj = driver.find_element_by_class_name('zV_Nj').text[4:-1].replace(',','') # 좋아요 수 구하기 
        print('좋아요: ',like_obj)
    #     like_float = float(like_obj)
    #     like = int(like_obj)

        time.sleep(1)
        driver.find_element_by_class_name('zV_Nj').click() # 좋아요 클릭 
        time.sleep(3)

        likers_list = []
        likers_dic = {}

        likers = driver.find_elements_by_class_name('FPmhX.notranslate.MBL3Z') 
        for j in tqdm(range(len(likers))):
            likerss = driver.find_elements_by_class_name('FPmhX.notranslate.MBL3Z')
            time.sleep(3)

            print(likerss[j].text)
            link_info = likerss[j].get_attribute('href') # 링크

            likers_list.append(link_info)
            likers_dic = likers_list



            j += 1

            if j % 6 == 0 :

                 # 페이지 다운
                endOfPage = driver.find_element_by_css_selector('body > div.RnEpo.Yx5HN > div > div').find_elements_by_tag_name('a')[-1]
                endOfPage.send_keys(Keys.PAGE_DOWN)
                time.sleep(3)
                continue

        real_list.append(likers_dic)


        num += 1
        
    like_df = pd.DataFrame(
        {
        "content_link":content_links,
        "likers":real_list
        }
    )

    like_df.to_csv('./liker_vanlialatte')

In [17]:
# 각 유저의 프로필 정보 -> id가 있는 경우 

followers_count = []
gesimul_count =[]
sogaes = []

    
def get_profile(user_ids):

    
    i = 0
    for i in range(len(user_ids)):
        user_profile = 'https://www.instagram.com/{}/'.format(user_ids[i])
        driver.get(user_profile)

        try: 

            gesimul = int(driver.find_element_by_class_name('g47SY ').text.replace(',',''))

            followers = \
            int(driver.find_element_by_css_selector('#react-root > section > main > div > header > section > ul > li:nth-child(2) > a > span').get_attribute("title").replace(',',''))

            sogae = driver.find_element_by_class_name('-vDIg').text.replace('\n',' ')
        #     profiles.append(user_ids[i])
            gesimul_count.append(gesimul)
            followers_count.append(followers)
            sogaes.append(sogae)
            
            print("ID: ", user_ids[i])
            print("게시물: ",gesimul)
            print("팔로워: ",followers)
            print("소개: ",sogae)
            print(" ")

            i += 1
        except :
            followers_count.append("NaN")
            gesimul_count.append("NaN")
            sogaes.append("NaN")
            
        


In [18]:
get_profile(user_ids)

<ipython-input-17-9b6db92509f1>:18: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  gesimul = int(driver.find_element_by_class_name('g47SY ').text.replace(',',''))
<ipython-input-17-9b6db92509f1>:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  int(driver.find_element_by_css_selector('#react-root > section > main > div > header > section > ul > li:nth-child(2) > a > span').get_attribute("title").replace(',',''))
<ipython-input-17-9b6db92509f1>:23: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  sogae = driver.find_element_by_class_name('-vDIg').text.replace('\n',' ')


ID:  solchan_i
게시물:  407
팔로워:  356
소개:  자두밭 신혼일기 식품점 솔찬&율이네 시골집에 놀러오세요🎈 . 예약문의: 에어비앤비 메시지 주시면 가장 빠르게 확인 가능합니다 . 🐥🐥🐥🐥🐥🐥🐥🐥🐥🐥 www.airbnb.com/h/farmstay-solbam
 
ID:  seoul_sup
게시물:  4181
팔로워:  14350
소개:  서울촌놈
 
ID:  coejeongseon888
게시물:  65
팔로워:  895
소개:  #에제르 경북 김천시 아포읍 황소길 76 에제르는 사회적 가치의 이념을 가지고 봉사와 실천으로 사회에 환원할 수 있도록 노력하는 사회적 기업입니다. #에제르 #에제르체험 #에제르카페 #에제르힐링가든 #에제르스쿨 #김천에제르 #구미에제르 인스타그램 활용시 부탁드립니다. www.ezeredu.com
 
ID:  instainkorea
게시물:  4474
팔로워:  40641
소개:  대한민국 대표 인스타그램 📕📷📽대한민국 전국을 간다~~🐱 odiya.kr
 
ID:  cij1492
게시물:  199
팔로워:  518
소개:  선일전통창호 김천최목수 집 수리/인테리어 안과 밖을 경계짓다-전통창호 010-2540-6214 #carpenter #woodworking #woodworker #wooddoor #architecture #desine #customdoors #전통살문 #전통창호 #목수 #선일창호 m.blog.naver.com/cij1492
 
ID:  instainkorea
게시물:  4474
팔로워:  40641
소개:  대한민국 대표 인스타그램 📕📷📽대한민국 전국을 간다~~🐱 odiya.kr
 
ID:  olivia_like_you
게시물:  2263
팔로워:  13670
소개:  seven
 
ID:  carpediemzzang
게시물:  866
팔로워:  14022
소개:  까르페디엠 "현재를 즐겨라" 세상의맛..나만의멋.. 그리고.

In [19]:
print(len(user_ids))
print(len(location_infos))
print(len(location_hrefs))
print(len(main_texts))
print(len(tags))
print(len(likes))
print(len(date_texts))
print(len(date_times))
print(len(date_titles))

38
38
38
38
7
38
38
0
0


### DataFrame

In [20]:
instagram_df = pd.DataFrame(
    {
         "id" : user_ids
        ,"location_info" : location_infos
        ,"location_href" : location_hrefs
        ,"main_text" : main_texts
        ,"hastag" : tag_lists
        ,"like" : likes
       ,"date_text" : date_texts
        #,"date_time" : date_times
       #,"date_title" : date_titles
    }
)

In [21]:
user_df = pd.DataFrame(
    {
          "user_id": user_ids
        , "Posts": gesimul_count
        , "Followers": followers_count
        , "user_profile" : sogaes
    }
)

In [22]:
user_df.drop_duplicates(inplace=True)

In [23]:
user_df.head()

,user_id,Posts,Followers,user_profile
0,solchan_i,407,356,자두밭 신혼일기 식품점 솔찬&율이네 시골집에 놀러오세요🎈 . 예...
1,seoul_sup,4181,14350,서울촌놈
2,coejeongseon888,65,895,#에제르 경북 김천시 아포읍 황소길 76 에제르는 사회적 가치의 이념을 가지고 봉사...
3,instainkorea,4474,40641,대한민국 대표 인스타그램 📕📷📽대한민국 전국을 간다~~...
4,cij1492,199,518,선일전통창호 김천최목수 집 수리/인테리어 안과 밖을 경계짓다-전통창호 010-254...


In [24]:
instagram_df.head(10)

,id,location_info,location_href,main_text,hastag,like,date_text
0,solchan_i,Gimcheon,https://www.instagram.com/explore/locations/26...,"solchan_i 🏡 오늘의 손님 맞이, 편안한 쉼_이 되시길🎀 . . #에어비앤비...","[에어비앤비, 솔과밤, 김천여행, 김천숙소, 김천숙박, 팜스테이]",55,2021년 11월 5일
1,seoul_sup,NULL,NULL,seoul_sup #김천무인텔 #김천호텔 #김천숙소 #김천숙박 #김천모텔 #김천아포...,"[김천무인텔, 김천호텔, 김천숙소, 김천숙박, 김천모텔, 김천아포모텔, 김천아포무인...",9,2020년 6월 12일
2,coejeongseon888,NULL,NULL,coejeongseon888 우리 에제르는 지금 장미 만발~♡ 장미로 꽃꽃이를 쫘악...,"[에제르, 에제르힐링가든, 에제르시크릿가든, 에제르평생교육원, 김천핫플레이스, 김천...",52,2020년 6월 9일
3,instainkorea,NULL,NULL,instainkorea #김천무인텔 #김천호텔 #김천숙소 #김천숙박 #김천모텔 #김...,"[김천무인텔, 김천호텔, 김천숙소, 김천숙박, 김천모텔, 김천아포모텔, 김천아포무인...",6,2020년 6월 5일
4,cij1492,제로모텔,https://www.instagram.com/explore/locations/11...,cij1492 코로나이후 처음으로 숙박 만실하나 싶었더니 대실이 하나 들어와버렸다 ...,"[김천숙박, 김천제로모텔, 김천무인텔, 직지사모텔, 제로모텔, 코로나, 코로나꺼져,...",19,2020년 5월 30일
5,instainkorea,NULL,NULL,instainkorea #김천무인텔#김천호텔#김처...,"[김천무인텔, 김천호텔, 김천숙소, 김...",3,2020년 5월 6일
6,olivia_like_you,NULL,NULL,olivia_like_you #김천무인텔#김천호텔#김천숙소#김천숙박#김천모텔#김천아...,"[김천무인텔, 김천호텔, 김천숙소, 김천숙박, 김천모텔, 김천아포모텔, 김천아포무인...",5,2020년 3월 16일
7,carpediemzzang,NULL,NULL,carpediemzzang #김천무인텔#김천호텔#김천숙소#김천숙박#김천모텔#김천아포...,"[김천무인텔, 김천호텔, 김천숙소, 김천숙박, 김천모텔, 김천아포모텔, 김천아포무인...",13,2020년 3월 3일
8,seoul_sup,NULL,NULL,seoul_sup #김천무인텔#김천호텔#김천숙소#김천숙박#김천모텔#김천아포모텔#김천...,"[김천무인텔, 김천호텔, 김천숙소, 김천숙박, 김천모텔, 김천아포모텔, 김천아포무인...",12,2020년 1월 21일
9,samdamong,직지사 (直指寺),https://www.instagram.com/explore/locations/29...,samdamong #김천힐링플레이스 👑 오늘의 피로를 여기서 날려버리고~ 활기찬 내...,"[김천힐링플레이스, 단체손님, 단체손님환영, 선수단환영, 선수단숙소추천, 선수단숙소...",31,2019년 12월 9일


### Concat

In [25]:
result = pd.concat([instagram_df,user_df], axis=1)
result

,id,location_info,location_href,main_text,hastag,like,date_text,user_id,Posts,Followers,user_profile
0,solchan_i,Gimcheon,https://www.instagram.com/explore/locations/26...,"solchan_i 🏡 오늘의 손님 맞이, 편안한 쉼_이 되시길🎀 . . #에어비앤비...","[에어비앤비, 솔과밤, 김천여행, 김천숙소, 김천숙박, 팜스테이]",55,2021년 11월 5일,solchan_i,407.0,356.0,자두밭 신혼일기 식품점 솔찬&율이네 시골집에 놀러오세요🎈 . 예...
1,seoul_sup,NULL,NULL,seoul_sup #김천무인텔 #김천호텔 #김천숙소 #김천숙박 #김천모텔 #김천아포...,"[김천무인텔, 김천호텔, 김천숙소, 김천숙박, 김천모텔, 김천아포모텔, 김천아포무인...",9,2020년 6월 12일,seoul_sup,4181.0,14350.0,서울촌놈
2,coejeongseon888,NULL,NULL,coejeongseon888 우리 에제르는 지금 장미 만발~♡ 장미로 꽃꽃이를 쫘악...,"[에제르, 에제르힐링가든, 에제르시크릿가든, 에제르평생교육원, 김천핫플레이스, 김천...",52,2020년 6월 9일,coejeongseon888,65.0,895.0,#에제르 경북 김천시 아포읍 황소길 76 에제르는 사회적 가치의 이념을 가지고 봉사...
3,instainkorea,NULL,NULL,instainkorea #김천무인텔 #김천호텔 #김천숙소 #김천숙박 #김천모텔 #김...,"[김천무인텔, 김천호텔, 김천숙소, 김천숙박, 김천모텔, 김천아포모텔, 김천아포무인...",6,2020년 6월 5일,instainkorea,4474.0,40641.0,대한민국 대표 인스타그램 📕📷📽대한민국 전국을 간다~~...
4,cij1492,제로모텔,https://www.instagram.com/explore/locations/11...,cij1492 코로나이후 처음으로 숙박 만실하나 싶었더니 대실이 하나 들어와버렸다 ...,"[김천숙박, 김천제로모텔, 김천무인텔, 직지사모텔, 제로모텔, 코로나, 코로나꺼져,...",19,2020년 5월 30일,cij1492,199.0,518.0,선일전통창호 김천최목수 집 수리/인테리어 안과 밖을 경계짓다-전통창호 010-254...
5,instainkorea,NULL,NULL,instainkorea #김천무인텔#김천호텔#김처...,"[김천무인텔, 김천호텔, 김천숙소, 김...",3,2020년 5월 6일,NaN,NaN,NaN,NaN
6,olivia_like_you,NULL,NULL,olivia_like_you #김천무인텔#김천호텔#김천숙소#김천숙박#김천모텔#김천아...,"[김천무인텔, 김천호텔, 김천숙소, 김천숙박, 김천모텔, 김천아포모텔, 김천아포무인...",5,2020년 3월 16일,olivia_like_you,2263.0,13670.0,seven
7,carpediemzzang,NULL,NULL,carpediemzzang #김천무인텔#김천호텔#김천숙소#김천숙박#김천모텔#김천아포...,"[김천무인텔, 김천호텔, 김천숙소, 김천숙박, 김천모텔, 김천아포모텔, 김천아포무인...",13,2020년 3월 3일,carpediemzzang,866.0,14022.0,"까르페디엠 ""현재를 즐겨라"" 세상의맛..나만의멋.. 그리고..사랑가득한 일상"
8,seoul_sup,NULL,NULL,seoul_sup #김천무인텔#김천호텔#김천숙소#김천숙박#김천모텔#김천아포모텔#김천...,"[김천무인텔, 김천호텔, 김천숙소, 김천숙박, 김천모텔, 김천아포모텔, 김천아포무인...",12,2020년 1월 21일,NaN,NaN,NaN,NaN
9,samdamong,직지사 (直指寺),https://www.instagram.com/explore/locations/29...,samdamong #김천힐링플레이스 👑 오늘의 피로를 여기서 날려버리고~ 활기찬 내...,"[김천힐링플레이스, 단체손님, 단체손님환영, 선수단환영, 선수단숙소추천, 선수단숙소...",31,2019년 12월 9일,samdamong,1354.0,1388.0,夢 개인 블로그


## To_csv

In [26]:
file_name = 'instagrame_1110_김천숙박(30)' ##### 파일명
file_path = 'C:/Users/seoyeonho/Desktop/Crawler_csv' ### 저장할 경로 

In [27]:
result.to_csv('{}{}.csv'.format(file_path,file_name), index=False, encoding='utf-8')

### Read

In [28]:
pd.read_csv('{}{}.csv'.format(file_path,file_name))

,id,location_info,location_href,main_text,hastag,like,date_text,user_id,Posts,Followers,user_profile
0,solchan_i,Gimcheon,https://www.instagram.com/explore/locations/26...,"solchan_i 🏡 오늘의 손님 맞이, 편안한 쉼_이 되시길🎀 . . #에어비앤비...","['에어비앤비', '솔과밤', '김천여행', '김천숙소', '김천숙박', '팜스테이']",55.0,2021년 11월 5일,solchan_i,407.0,356.0,자두밭 신혼일기 식품점 솔찬&율이네 시골집에 놀러오세요🎈 . 예...
1,seoul_sup,NaN,NaN,seoul_sup #김천무인텔 #김천호텔 #김천숙소 #김천숙박 #김천모텔 #김천아포...,"['김천무인텔', '김천호텔', '김천숙소', '김천숙박', '김천모텔', '김천아...",9.0,2020년 6월 12일,seoul_sup,4181.0,14350.0,서울촌놈
2,coejeongseon888,NaN,NaN,coejeongseon888 우리 에제르는 지금 장미 만발~♡ 장미로 꽃꽃이를 쫘악...,"['에제르', '에제르힐링가든', '에제르시크릿가든', '에제르평생교육원', '김천...",52.0,2020년 6월 9일,coejeongseon888,65.0,895.0,#에제르 경북 김천시 아포읍 황소길 76 에제르는 사회적 가치의 이념을 가지고 봉사...
3,instainkorea,NaN,NaN,instainkorea #김천무인텔 #김천호텔 #김천숙소 #김천숙박 #김천모텔 #김...,"['김천무인텔', '김천호텔', '김천숙소', '김천숙박', '김천모텔', '김천아...",6.0,2020년 6월 5일,instainkorea,4474.0,40641.0,대한민국 대표 인스타그램 📕📷📽대한민국 전국을 간다~~...
4,cij1492,제로모텔,https://www.instagram.com/explore/locations/11...,cij1492 코로나이후 처음으로 숙박 만실하나 싶었더니 대실이 하나 들어와버렸다 ...,"['김천숙박', '김천제로모텔', '김천무인텔', '직지사모텔', '제로모텔', '...",19.0,2020년 5월 30일,cij1492,199.0,518.0,선일전통창호 김천최목수 집 수리/인테리어 안과 밖을 경계짓다-전통창호 010-254...
5,instainkorea,NaN,NaN,instainkorea #김천무인텔#김천호텔#김처...,"['김천무인텔', '김천호텔', '김천숙소...",3.0,2020년 5월 6일,NaN,NaN,NaN,NaN
6,olivia_like_you,NaN,NaN,olivia_like_you #김천무인텔#김천호텔#김천숙소#김천숙박#김천모텔#김천아...,"['김천무인텔', '김천호텔', '김천숙소', '김천숙박', '김천모텔', '김천아...",5.0,2020년 3월 16일,olivia_like_you,2263.0,13670.0,seven
7,carpediemzzang,NaN,NaN,carpediemzzang #김천무인텔#김천호텔#김천숙소#김천숙박#김천모텔#김천아포...,"['김천무인텔', '김천호텔', '김천숙소', '김천숙박', '김천모텔', '김천아...",13.0,2020년 3월 3일,carpediemzzang,866.0,14022.0,"까르페디엠 ""현재를 즐겨라"" 세상의맛..나만의멋.. 그리고..사랑가득한 일상"
8,seoul_sup,NaN,NaN,seoul_sup #김천무인텔#김천호텔#김천숙소#김천숙박#김천모텔#김천아포모텔#김천...,"['김천무인텔', '김천호텔', '김천숙소', '김천숙박', '김천모텔', '김천아...",12.0,2020년 1월 21일,NaN,NaN,NaN,NaN
9,samdamong,직지사 (直指寺),https://www.instagram.com/explore/locations/29...,samdamong #김천힐링플레이스 👑 오늘의 피로를 여기서 날려버리고~ 활기찬 내...,"['김천힐링플레이스', '단체손님', '단체손님환영', '선수단환영', '선수단숙소...",31.0,2019년 12월 9일,samdamong,1354.0,1388.0,夢 개인 블로그


In [29]:
instagram_df.to_csv('{}'.format("0908.csv"), index=False, encoding='utf-8')

In [30]:
instagram_df

,id,location_info,location_href,main_text,hastag,like,date_text
0,solchan_i,Gimcheon,https://www.instagram.com/explore/locations/26...,"solchan_i 🏡 오늘의 손님 맞이, 편안한 쉼_이 되시길🎀 . . #에어비앤비...","[에어비앤비, 솔과밤, 김천여행, 김천숙소, 김천숙박, 팜스테이]",55,2021년 11월 5일
1,seoul_sup,NULL,NULL,seoul_sup #김천무인텔 #김천호텔 #김천숙소 #김천숙박 #김천모텔 #김천아포...,"[김천무인텔, 김천호텔, 김천숙소, 김천숙박, 김천모텔, 김천아포모텔, 김천아포무인...",9,2020년 6월 12일
2,coejeongseon888,NULL,NULL,coejeongseon888 우리 에제르는 지금 장미 만발~♡ 장미로 꽃꽃이를 쫘악...,"[에제르, 에제르힐링가든, 에제르시크릿가든, 에제르평생교육원, 김천핫플레이스, 김천...",52,2020년 6월 9일
3,instainkorea,NULL,NULL,instainkorea #김천무인텔 #김천호텔 #김천숙소 #김천숙박 #김천모텔 #김...,"[김천무인텔, 김천호텔, 김천숙소, 김천숙박, 김천모텔, 김천아포모텔, 김천아포무인...",6,2020년 6월 5일
4,cij1492,제로모텔,https://www.instagram.com/explore/locations/11...,cij1492 코로나이후 처음으로 숙박 만실하나 싶었더니 대실이 하나 들어와버렸다 ...,"[김천숙박, 김천제로모텔, 김천무인텔, 직지사모텔, 제로모텔, 코로나, 코로나꺼져,...",19,2020년 5월 30일
5,instainkorea,NULL,NULL,instainkorea #김천무인텔#김천호텔#김처...,"[김천무인텔, 김천호텔, 김천숙소, 김...",3,2020년 5월 6일
6,olivia_like_you,NULL,NULL,olivia_like_you #김천무인텔#김천호텔#김천숙소#김천숙박#김천모텔#김천아...,"[김천무인텔, 김천호텔, 김천숙소, 김천숙박, 김천모텔, 김천아포모텔, 김천아포무인...",5,2020년 3월 16일
7,carpediemzzang,NULL,NULL,carpediemzzang #김천무인텔#김천호텔#김천숙소#김천숙박#김천모텔#김천아포...,"[김천무인텔, 김천호텔, 김천숙소, 김천숙박, 김천모텔, 김천아포모텔, 김천아포무인...",13,2020년 3월 3일
8,seoul_sup,NULL,NULL,seoul_sup #김천무인텔#김천호텔#김천숙소#김천숙박#김천모텔#김천아포모텔#김천...,"[김천무인텔, 김천호텔, 김천숙소, 김천숙박, 김천모텔, 김천아포모텔, 김천아포무인...",12,2020년 1월 21일
9,samdamong,직지사 (直指寺),https://www.instagram.com/explore/locations/29...,samdamong #김천힐링플레이스 👑 오늘의 피로를 여기서 날려버리고~ 활기찬 내...,"[김천힐링플레이스, 단체손님, 단체손님환영, 선수단환영, 선수단숙소추천, 선수단숙소...",31,2019년 12월 9일
